In [1]:
using JLD2, CSV, DataFrames
using KissSmoothing
using Statistics
using Plots

In [ ]:
ndays = 4
saveat = 60.0
dt = 60 # minutes
nruns = 000
n_species = 5946
nmete = 5
ntimestep = 24*ndays +1
startspec = 1
specname =  Array(CSV.read("specname.csv", DataFrame))[startspec:n_species,2]

5946-element Vector{Any}:
 "so4_conc_1"
 "so4_conc_2"
 "so4_conc_3"
 "no3_conc_1"
 "no3_conc_2"
 "no3_conc_3"
 "cl_conc_1"
 "cl_conc_2"
 "cl_conc_3"
 "nh4_conc_1"
 "nh4_conc_2"
 "nh4_conc_3"
 "msa_conc_1"
 ⋮
 "HOCH2CHO"
 "MEKAO2"
 "C116CHO"
 "CH3COCH2O2"
 "C3ME3CHOO2"
 "CH3CO3"
 "C1011CO3"
 "IPRHOCO3"
 "C132O2"
 "BCLKET"
 "BCAL"
 "BCKET"

In [ ]:
JLD2.@load "training_dataset.jld" conc_train emit_train mete_train id_train


4-element Vector{Symbol}:
 :conc_train
 :emit_train
 :mete_train
 :id_train

In [4]:
discard_specname = vcat(string.(specname[1:60]),["H2SO4", "ETHP", "ANOL", "ETHOOH", "ALD2", "RCOOH", "C2O3", "ARO1", "ARO2", "ALK1", "OLE1", "API1", "API2", "LIM1", "LIM2", "PAR", "AONE", "MGLY", "ETH", "OLET", "OLEI", "TOL", "XYL", "CRES", "TO2", "CRO", "OPEN", "ONIT", "ROOH", "RO2", "ANO2", "NAP", "XO2", "XPAR", "ISOP", "ISOPRD", "ISOPP", "ISOPN", "ISOPO2", "API", "LIM", "CH3SO2H", "CH3SCH2OO", "CH3SO2OO", "CH3SO2CH2OO", "SULFHOX", "NA", "SA"])

108-element Vector{String}:
 "so4_conc_1"
 "so4_conc_2"
 "so4_conc_3"
 "no3_conc_1"
 "no3_conc_2"
 "no3_conc_3"
 "cl_conc_1"
 "cl_conc_2"
 "cl_conc_3"
 "nh4_conc_1"
 "nh4_conc_2"
 "nh4_conc_3"
 "msa_conc_1"
 ⋮
 "ISOPP"
 "ISOPN"
 "ISOPO2"
 "API"
 "LIM"
 "CH3SO2H"
 "CH3SCH2OO"
 "CH3SO2OO"
 "CH3SO2CH2OO"
 "SULFHOX"
 "NA"
 "SA"

In [5]:
# Get indices of s in S
indices_s = findall(x -> x in discard_specname, specname)

# Get all indices of S
all_indices = collect(1:length(specname))

# Get indices that are not in s
non_s_indices = setdiff(all_indices, indices_s)

# Output the result
#println(non_s_indices)

5838-element Vector{Int64}:
   62
   63
   64
   65
   66
   67
   68
   69
   70
   71
   72
   73
   74
    ⋮
 5935
 5936
 5937
 5938
 5939
 5940
 5941
 5942
 5943
 5944
 5945
 5946

In [6]:
specname = specname[non_s_indices]

5838-element Vector{Any}:
 "HNO3"
 "HCL"
 "NH3"
 "NO"
 "NO2"
 "NO3"
 "N2O5"
 "HONO"
 "HNO4"
 "O3"
 "O1D"
 "O3P"
 "OH"
 ⋮
 "HOCH2CHO"
 "MEKAO2"
 "C116CHO"
 "CH3COCH2O2"
 "C3ME3CHOO2"
 "CH3CO3"
 "C1011CO3"
 "IPRHOCO3"
 "C132O2"
 "BCLKET"
 "BCAL"
 "BCKET"

In [7]:
ref_data = conc_train[:,1:ntimestep,:][non_s_indices,:,:]
ref_mean = mean(ref_data;dims=(2,3))

5838×1×1 Array{Float64, 3}:
[:, :, 1] =
 41.830507727364285
  0.826083234230604
 10.426867268653771
 73.5211872700622
 87.05575220817245
  4.497413780249559e-5
  0.1288496116352216
  0.2614451969039195
  0.0
 34.70999811556409
  1.0421094771391515e-12
  0.0
  2.425822497601047e-5
  ⋮
  0.47886239450799495
  9.273242404065054e-8
  1.0593779906314397e-5
  5.011332135180595e-7
  1.989689754937701e-9
  3.610796252990747e-6
  1.6213221108056465e-10
  1.899497503825966e-10
  1.0870381789732724e-9
  9.468119667579426e-5
  5.4324010737429565e-5
  2.8423785189251166e-8

In [ ]:

ref_emit = emit_train[:,1:ntimestep,:][non_s_indices,:,:]

5838×97×4350 Array{Float64, 3}:
[:, :, 1] =
 0.0         0.0          0.0          …  0.0          0.0
 0.0         0.0          0.0             0.0          0.0
 5.73522e-5  0.0          0.0             8.60284e-5   5.73522e-5
 0.00185247  0.00129267   0.00125149      0.00199244   0.00185247
 0.0002089   0.000145769  0.000141122     0.000224688  0.0002089
 0.0         0.0          0.0          …  0.0          0.0
 0.0         0.0          0.0             0.0          0.0
 0.0         0.0          0.0             0.0          0.0
 0.0         0.0          0.0             0.0          0.0
 0.0         0.0          0.0             0.0          0.0
 0.0         0.0          0.0          …  0.0          0.0
 0.0         0.0          0.0             0.0          0.0
 0.0         0.0          0.0             0.0          0.0
 ⋮                                     ⋱  ⋮            
 0.0         0.0          0.0             0.0          0.0
 0.0         0.0          0.0             0.0         

In [9]:
conc_train = nothing
emit_train = nothing
GC.gc()

In [10]:
begin
plots = []
i_case = 301
for i in 1:15
    p = plot(ref_data[i,:,i_case], title = specname[i], legend = :none )
    push!(plots,p)
end
plot(plots...)
end

In [11]:
ref_data[ref_data.< 0.0] .= 0.0
GC.gc()

In [ ]:

ref_params = mete_train[:,1:ntimestep,:]

5×97×4350 Array{Float64, 3}:
[:, :, 1] =
  54.4       54.4       54.4      …   54.4       54.4       54.4
 275.502    275.502    275.502       275.502    275.502    275.502
  87.72      87.72      87.72         87.72      87.72      87.72
   1.81804    1.8371     1.81771       1.68819    1.77647    1.83269
   1.06595    1.06595    1.06595       1.06595    1.06595    1.06595

[:, :, 2] =
  55.42      55.42      55.42     …   55.42      55.42      55.42
 294.688    294.688    294.688       294.688    294.688    294.688
  83.98      83.98      83.98         83.98      83.98      83.98
   2.04811    2.06982    2.04798       1.91439    2.01174    2.0752
   1.02943    1.02943    1.02943       1.02943    1.02943    1.02943

[:, :, 3] =
  64.91      64.91      64.91     …   64.91      64.91      64.91
 266.112    266.112    266.112       266.112    266.112    266.112
  74.27      74.27      74.27         74.27      74.27      74.27
   2.18718    2.20464    2.18714       2.08336    2.16061    2

In [13]:

timelength = 60 * (ndays * 24) # minutes
times = LinRange(0, timelength, ntimestep)
#times = LinRange(0 , timelength, Int((timelength) / dt) + 1)
tspan = (times[1], times[end])

(0.0, 5760.0)

In [14]:
ref_data

5838×97×4350 Array{Float64, 3}:
[:, :, 1] =
 35.5621       35.8408       36.0357       …  35.5734       35.9289
  0.802068      0.803382      0.804595         0.802582      0.803909
 10.6593       10.3227        9.80636         10.9235       10.6752
 79.8003       76.7382       72.6226          82.6934       79.583
 87.9366       87.078        85.4128          88.5921       88.0784
  2.1578e-5     2.27929e-5    2.44998e-5   …   2.05207e-5    2.1723e-5
  0.0302629     0.0316757     0.0333736        0.0289671     0.0305198
  0.288014      0.270394      0.25303          0.305858      0.286391
  0.0           0.0           0.0              0.0           0.0
 28.5783       29.3545       30.4999          27.9178       28.6481
  1.0267e-12    1.04337e-12   1.09593e-12  …   1.02386e-12   1.02085e-12
  0.0           0.0           0.0              0.0           0.0
  2.12513e-5    2.07019e-5    2.05173e-5       2.1893e-5     2.11445e-5
  ⋮                                        ⋱   ⋮            

In [15]:
using DataInterpolations

In [16]:
ref_data_train = ref_data
ref_emit_train = ref_emit
ref_params_train = ref_params

5×97×4350 Array{Float64, 3}:
[:, :, 1] =
  54.4       54.4       54.4      …   54.4       54.4       54.4
 275.502    275.502    275.502       275.502    275.502    275.502
  87.72      87.72      87.72         87.72      87.72      87.72
   1.81804    1.8371     1.81771       1.68819    1.77647    1.83269
   1.06595    1.06595    1.06595       1.06595    1.06595    1.06595

[:, :, 2] =
  55.42      55.42      55.42     …   55.42      55.42      55.42
 294.688    294.688    294.688       294.688    294.688    294.688
  83.98      83.98      83.98         83.98      83.98      83.98
   2.04811    2.06982    2.04798       1.91439    2.01174    2.0752
   1.02943    1.02943    1.02943       1.02943    1.02943    1.02943

[:, :, 3] =
  64.91      64.91      64.91     …   64.91      64.91      64.91
 266.112    266.112    266.112       266.112    266.112    266.112
  74.27      74.27      74.27         74.27      74.27      74.27
   2.18718    2.20464    2.18714       2.08336    2.16061    2

In [17]:
JLD2.jldsave("training_set.jld"; ref_data_train, ref_emit_train, ref_params_train, specname)